In [40]:
# Sử dụng GPU nếu có
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Tham số mô hình
vocab_size = wordEmbeddings.shape[0]
embedding_dim = wordEmbeddings.shape[1]
hidden_dim = 128  # Bạn có thể điều chỉnh
num_labels = len(label2Idx)
dropout = 0.3
seed = 9
epochs = 50  
# Tạo mô hình
model = biLSTM(
    vocab_size=vocab_size,
    input_size=embedding_dim,
    hidden_size=hidden_dim,
    output_size=num_labels,
    word_embeddings=wordEmbeddings,
    dropout=dropout,
    device=device,
    seed=9, 
)

model.to(device)

# Định nghĩa loss function và optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.001)

Using device: cuda


In [42]:
train_model_baseline(
    model=model,
    train_loaders=train_loaders,
    dev_loaders=dev_loaders,
    optimizer=optimizer,
    criterion=criterion,
    epochs=epochs,name_model="biLSTM"
)

Epoch 1/50, Loss: 0.4686
Train Precision: 0.7731, Recall: 0.6236, F1: 0.6575
Dev Precision: 0.7806, Recall: 0.5948, F1: 0.6372

Epoch 2/50, Loss: 0.1948
Train Precision: 0.7713, Recall: 0.7764, F1: 0.7639
Dev Precision: 0.7613, Recall: 0.7561, F1: 0.7490

Epoch 3/50, Loss: 0.1410
Train Precision: 0.7960, Recall: 0.8204, F1: 0.7991
Dev Precision: 0.7793, Recall: 0.7962, F1: 0.7799

Epoch 4/50, Loss: 0.1132
Train Precision: 0.8643, Recall: 0.8382, F1: 0.8466
Dev Precision: 0.8338, Recall: 0.8011, F1: 0.8126

Epoch 5/50, Loss: 0.0938
Train Precision: 0.9151, Recall: 0.8427, F1: 0.8726
Dev Precision: 0.8844, Recall: 0.7972, F1: 0.8324

Epoch 6/50, Loss: 0.0797
Train Precision: 0.9288, Recall: 0.8634, F1: 0.8919
Dev Precision: 0.8905, Recall: 0.8112, F1: 0.8450

Epoch 7/50, Loss: 0.0689
Train Precision: 0.9370, Recall: 0.8832, F1: 0.9070
Dev Precision: 0.8967, Recall: 0.8193, F1: 0.8522

Epoch 8/50, Loss: 0.0593
Train Precision: 0.9511, Recall: 0.8941, F1: 0.9203
Dev Precision: 0.9047, Reca

In [12]:
from sklearn.metrics import classification_report

In [50]:
model.eval()
all_preds = []
all_labels = []
    
with torch.no_grad():
    for loader in test_loaders:
        for batch in loader:
            tokens, labels = batch
            tokens = tokens.to(model.device)
            labels = labels.to(model.device)
                
            outputs = model.forward(tokens)  # [batch_size, seq_len, num_labels]
            preds = torch.argmax(outputs, dim=-1)  # [batch_size, seq_len]
                
            all_preds.extend(preds.cpu().numpy().tolist())
            all_labels.extend(labels.cpu().numpy().tolist())
    
y_pred = []
y_true = []
for pred_seq, true_seq in zip(all_preds, all_labels):
    for p, t in zip(pred_seq, true_seq):
            y_pred.append(p)
            y_true.append(t)
labels = list(label2Idx.values())
target_names = list(label2Idx.keys())
report = classification_report(y_true, y_pred, labels=labels, target_names=target_names, zero_division=0)
print(report)

              precision    recall  f1-score   support

       B-LOC       0.88      0.91      0.90      1668
      B-MISC       0.81      0.72      0.76       702
       B-ORG       0.84      0.82      0.83      1661
       B-PER       0.95      0.91      0.93      1617
       I-LOC       0.80      0.77      0.79       257
      I-MISC       0.68      0.55      0.61       216
       I-ORG       0.82      0.70      0.76       835
       I-PER       0.98      0.95      0.96      1156
           O       0.98      0.99      0.99     38323

    accuracy                           0.96     46435
   macro avg       0.86      0.81      0.84     46435
weighted avg       0.96      0.96      0.96     46435

